# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096992


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.40s/it]

Rendering models:  17%|█▋        | 5/30 [00:02<00:24,  1.01it/s]

Rendering models:  23%|██▎       | 7/30 [00:03<00:16,  1.40it/s]

Rendering models:  30%|███       | 9/30 [00:05<00:18,  1.11it/s]

...

Rendering models:  67%|██████▋   | 20/30 [00:07<00:04,  2.28it/s]

Rendering models:  80%|████████  | 24/30 [00:07<00:01,  3.12it/s]

Rendering models:  90%|█████████ | 27/30 [00:07<00:00,  4.26it/s]

Rendering models: 100%|██████████| 30/30 [00:07<00:00,  5.57it/s]

ElisabethB                            0.000179
Jnursssmith5263                       0.001819
DannyU                                0.000106
Jonas_Cross                           0.000693
awright5                              0.001564
whitelaughter                         0.000142
jaksonA                               0.105263
salmi.a01                             0.001209
not-logged-in-c6917d9b19f41e6c2995    0.170268
rsmith8235                            0.000232
DrPZ                                  0.000199
icelegance                            0.000310
clairedeu                             0.000112
aidenr                                0.000338
Dpwillet                              0.000431
Lavadude                              0.006017
ShirleyL                              0.000133
w7250369                              0.001029
Tessaje04                             2.175062
Bambura                               0.000481
fanboy_deluxe                         0.000627
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())